In [ ]:
from __future__ import annotations
import sys

# ---------- Pretty output ----------
try:
    import curses
    curses.setupterm()
    assert curses.tigetnum("colors") > 2
    OK   = "\x1b[1;%dm[ OK ]\x1b[0m"   % (30 + curses.COLOR_GREEN)
    FAIL = "\x1b[1;%dm[FAIL]\x1b[0m"   % (30 + curses.COLOR_RED)
except Exception:
    OK = "[ OK ]"
    FAIL = "[FAIL]"

# ---------- Version helpers ----------
from importlib import import_module, metadata
from packaging.version import Version

# Map de nombre de módulo -> nombre de distribución (para metadata)
PKG_TO_DIST = {
    "sklearn": "scikit-learn",
    "PIL": "Pillow",
    "IPython": "ipython",
    # los demás suelen coincidir
}

def get_dist_name(pkg: str) -> str:
    return PKG_TO_DIST.get(pkg, pkg)

def import_and_check(pkg: str, min_ver_str: str, fail_msg: str = ""):
    """Importa el módulo `pkg`, obtiene su versión de la distribución instalada
    y la compara con `min_ver_str`."""
    try:
        mod = import_module(pkg)
    except ImportError:
        print(FAIL, f"{pkg} no está instalado. {fail_msg}")
        return None

    dist_name = get_dist_name(pkg)
    try:
        ver_str = metadata.version(dist_name)
    except metadata.PackageNotFoundError:
        # fallback: intenta leer __version__ del módulo
        ver_str = getattr(mod, "__version__", None)
        if not ver_str:
            print(FAIL, f"No se pudo determinar la versión de {pkg}. {fail_msg}")
            return mod

    try:
        installed = Version(ver_str)
        required  = Version(min_ver_str)
    except Exception:
        print(FAIL, f"Versión inválida para {pkg}: '{ver_str}'.")
        return mod

    if installed < required:
        print(FAIL, f"{pkg} requiere >= {required}, pero hay {installed}.")
    else:
        print(OK, f"{pkg} versión {installed}")

    return mod

# ---------- Python version ----------
print("Using python in", sys.prefix)
print(sys.version)
py = sys.version_info
if not ((py.major == 3 and (py.minor >= 4)) or (py.major == 2 and py.minor >= 7)):
    print(FAIL, f"Se requiere Python 3.4+ (o 2.7), pero hay {sys.version}")

print()

# ---------- Requisitos ----------
requirements = {
    "numpy": "1.6.1",
    "scipy": "0.9",
    "matplotlib": "1.0",
    "IPython": "3.0",
    "sklearn": "0.18",
    "pandas": "0.19",
}

for pkg, min_ver in requirements.items():
    import_and_check(pkg, min_ver)
